# 🏥 EHR-M-GAN Training on Google Colab

**Version:** 2.0 (Production-Ready)

**Objectif:** Générer des données médicales synthétiques de haute qualité

**Dataset:** eICU-CRD Demo (1,650 patients)

---

## ⚙️ Configuration

**GPU recommandé:** T4 ou A100

**Runtime:** GPU (obligatoire)

**Temps estimé:** 6-8 heures total

## 📦 Étape 1 : Setup Initial

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Vérifier accès
!ls "/content/drive/MyDrive/" | head -10

In [ ]:
# Vérifier GPU disponible
!nvidia-smi

In [ ]:
# Cloner le repository
import os

# Supprimer si existe déjà
!rm -rf /content/ehrMGAN

# Cloner
!git clone https://github.com/jli0117/ehrMGAN.git /content/ehrMGAN

# Se placer dans le dossier
%cd /content/ehrMGAN

# Vérifier structure
!ls -la

## 🔧 Étape 2 : Installation TensorFlow 1.15

In [ ]:
# CRITIQUE : Downgrade vers TensorFlow 1.15
!pip uninstall -y tensorflow tensorflow-gpu -q
!pip install tensorflow-gpu==1.15.5 -q

# Vérifier version
import tensorflow as tf
print(f"✅ TensorFlow version: {tf.__version__}")
assert tf.__version__.startswith('1.15'), "❌ TensorFlow 1.15 requis!"

## 📚 Étape 3 : Installer Dépendances

In [ ]:
# Installer toutes les dépendances (versions testées)
!pip install --upgrade pip setuptools wheel -q

# Core dependencies
!pip install numpy==1.19.5 -q
!pip install pandas==1.1.5 -q
!pip install scipy==1.5.4 -q
!pip install scikit-learn==0.24.2 -q
!pip install matplotlib==3.3.4 -q
!pip install seaborn==0.11.2 -q
!pip install h5py==2.10.0 -q
!pip install tqdm==4.64.1 -q
!pip install pyyaml==5.4.1 -q

# PyTorch (pour contrastive loss)
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 -q

# Vérifier imports
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
print("✅ Toutes les dépendances installées")

## 📁 Étape 4 : Préparer les Données

In [ ]:
# Configuration des chemins
import shutil

# Chemins Drive (à adapter selon votre structure)
DRIVE_DATA = "/content/drive/MyDrive/ehrMGAN_data/eicu-crd-demo-2.0.1"
DRIVE_FIXES = "/content/drive/MyDrive/ehrMGAN_fixes"
DRIVE_CHECKPOINT = "/content/drive/MyDrive/ehrMGAN_checkpoints"

# Chemins locaux
LOCAL_DATA = "/content/ehrMGAN/preprocessing_physionet-main/eicu_preprocess/data"

# Créer dossiers
os.makedirs(LOCAL_DATA, exist_ok=True)
os.makedirs(DRIVE_CHECKPOINT, exist_ok=True)

print("✅ Dossiers créés")

In [ ]:
# Copier données eICU depuis Drive
required_files = [
    "patient.csv.gz",
    "vitalPeriodic.csv.gz",
    "infusionDrug.csv.gz",
    "respiratoryCare.csv.gz"
]

print("Copie des données eICU...")
for file in required_files:
    src = os.path.join(DRIVE_DATA, file)
    dst = os.path.join(LOCAL_DATA, file)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print(f"  ✅ {file}")
    else:
        print(f"  ❌ MANQUANT : {file}")
        print(f"     Téléchargez depuis PhysioNet et uploadez dans Drive")

# Vérifier
!ls -lh {LOCAL_DATA}

In [ ]:
# Copier fichiers de fix depuis Drive
print("Application des fixes...")

# visualise.py
if os.path.exists(f"{DRIVE_FIXES}/visualise.py"):
    !cp "{DRIVE_FIXES}/visualise.py" /content/ehrMGAN/evaluation_metrics/
    print("  ✅ visualise.py")
else:
    print("  ⚠️  visualise.py manquant (pourrait causer erreur)")

# utils/
if os.path.exists(f"{DRIVE_FIXES}/utils"):
    !cp -r "{DRIVE_FIXES}/utils/"*.py /content/ehrMGAN/preprocessing_physionet-main/eicu_preprocess/utils/
    print("  ✅ utils/*.py")

# preprocessing_eicu_complete.py
if os.path.exists(f"{DRIVE_FIXES}/preprocessing_eicu_complete.py"):
    !cp "{DRIVE_FIXES}/preprocessing_eicu_complete.py" /content/ehrMGAN/preprocessing_physionet-main/eicu_preprocess/
    print("  ✅ preprocessing_eicu_complete.py")

print("\n✅ Fixes appliqués")

## 🔄 Étape 5 : Preprocessing

In [ ]:
# Se placer dans le dossier preprocessing
%cd /content/ehrMGAN/preprocessing_physionet-main/eicu_preprocess

# Lancer preprocessing
!python preprocessing_eicu_complete.py \
  --data_path ./data \
  --output_path ../../data/real/eicu \
  --time_window 24 \
  --min_length 12 \
  --max_length 240 \
  --age_min 18 \
  --verbose

# Vérifier outputs
print("\n" + "="*50)
print("Fichiers générés :")
!ls -lh ../../data/real/eicu/

In [ ]:
# OPTIONNEL : Fix pickle protocol si erreur
import pickle

def convert_pickle_protocol(file_path):
    """Convertir pickle protocol 5 → 4"""
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        with open(file_path, 'wb') as f:
            pickle.dump(data, f, protocol=4)
        print(f"✅ Converti : {file_path}")
    except Exception as e:
        print(f"❌ Erreur : {e}")

# Convertir tous les .pkl
pkl_files = [
    "../../data/real/eicu/vital_sign_24hrs.pkl",
    "../../data/real/eicu/med_interv_24hrs.pkl",
    "../../data/real/eicu/statics.pkl"
]

for pkl_file in pkl_files:
    if os.path.exists(pkl_file):
        convert_pickle_protocol(pkl_file)

## 🎓 Étape 6 : Configuration Training

In [ ]:
# Revenir au dossier principal
%cd /content/ehrMGAN

# Paramètres optimisés pour Colab
BATCH_SIZE = 128          # Réduit pour éviter OOM
NUM_PRE_EPOCHS = 500      # Pretraining VAE
NUM_EPOCHS = 800          # Training adversarial
CHECKPOINT_FREQ = 50      # Sauvegardes fréquentes
Z_DIM = 25                # Dimension latente

# Créer dossiers de sortie
!mkdir -p data/checkpoint
!mkdir -p data/fake
!mkdir -p logs/visualizations

print(f"""Configuration Training:
- Batch Size: {BATCH_SIZE}
- Pretraining Epochs: {NUM_PRE_EPOCHS}
- Training Epochs: {NUM_EPOCHS}
- Checkpoint Freq: {CHECKPOINT_FREQ}
- Latent Dim: {Z_DIM}
""")

In [ ]:
# Activer anti-déconnexion Colab
from IPython.display import display, Javascript

display(Javascript('''
    function KeepClicking(){
        console.log("Keeping session alive");
        document.querySelector("colab-toolbar-button#connect").click();
    }
    setInterval(KeepClicking, 60000);
'''))

print("✅ Anti-déconnexion activé (click toutes les 60s)")

## 🚀 Étape 7 : Phase 1 - Pretraining VAE

In [ ]:
# Lancer pretraining VAE
!python main_train.py \
  --dataset eicu \
  --data_path ./data/real/eicu \
  --batch_size {BATCH_SIZE} \
  --num_pre_epochs {NUM_PRE_EPOCHS} \
  --num_epochs 0 \
  --epoch_ckpt_freq {CHECKPOINT_FREQ} \
  --z_dim {Z_DIM} \
  --conditional False

print("\n" + "="*50)
print("✅ Pretraining VAE terminé")

In [ ]:
# Sauvegarder checkpoint pretraining dans Drive
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint_name = f"pretraining_complete_{timestamp}"
checkpoint_dir = f"{DRIVE_CHECKPOINT}/{checkpoint_name}"

shutil.copytree("data/checkpoint", checkpoint_dir, dirs_exist_ok=True)
print(f"✅ Checkpoint sauvegardé : {checkpoint_dir}")

## 🔥 Étape 8 : Phase 2 - Training Adversarial

In [ ]:
# Lancer training GAN
!python main_train.py \
  --dataset eicu \
  --data_path ./data/real/eicu \
  --batch_size {BATCH_SIZE} \
  --num_pre_epochs 0 \
  --num_epochs {NUM_EPOCHS} \
  --epoch_ckpt_freq {CHECKPOINT_FREQ} \
  --z_dim {Z_DIM} \
  --conditional False \
  --resume_training True

print("\n" + "="*50)
print("✅ Training GAN terminé")

In [ ]:
# Sauvegarder résultats finaux dans Drive
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_export = f"{DRIVE_CHECKPOINT}/FINAL_EXPORT_{timestamp}"
os.makedirs(final_export, exist_ok=True)

# Copier tout
shutil.copytree("data/checkpoint", f"{final_export}/checkpoints", dirs_exist_ok=True)
shutil.copytree("data/fake", f"{final_export}/synthetic_data", dirs_exist_ok=True)
shutil.copytree("logs", f"{final_export}/logs", dirs_exist_ok=True)

print(f"✅ Export final complet : {final_export}")

## 📊 Étape 9 : Validation Résultats

In [ ]:
# Charger données synthétiques
import pickle

with open("data/fake/c_gen_data.pkl", "rb") as f:
    c_gen_data = pickle.load(f)

with open("data/fake/d_gen_data.pkl", "rb") as f:
    d_gen_data = pickle.load(f)

# Charger données réelles
with open("data/real/eicu/vital_sign_24hrs.pkl", "rb") as f:
    c_real_data = pickle.load(f)

with open("data/real/eicu/med_interv_24hrs.pkl", "rb") as f:
    d_real_data = pickle.load(f)

print(f"Synthétiques - Continues: {c_gen_data.shape}, Discrètes: {d_gen_data.shape}")
print(f"Réelles - Continues: {c_real_data.shape}, Discrètes: {d_real_data.shape}")

In [ ]:
# Visualiser comparaisons
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

num_samples = 5
feature_names = ['Heart Rate', 'SpO2', 'SBP', 'DBP', 'Temp', 'Resp Rate', 'GCS']

fig, axes = plt.subplots(num_samples, 2, figsize=(14, 10))

for i in range(num_samples):
    # Réel
    axes[i, 0].plot(c_real_data[i], alpha=0.7, linewidth=0.8)
    axes[i, 0].set_title(f"Patient {i+1} - Réel")
    axes[i, 0].set_ylabel("Valeur normalisée")
    
    # Synthétique
    axes[i, 1].plot(c_gen_data[i], alpha=0.7, linewidth=0.8)
    axes[i, 1].set_title(f"Patient {i+1} - Synthétique")
    
    if i == num_samples - 1:
        axes[i, 0].set_xlabel("Heure")
        axes[i, 1].set_xlabel("Heure")

plt.tight_layout()
plt.savefig(f"{DRIVE_CHECKPOINT}/comparison_trajectories.png", dpi=150)
plt.show()

In [ ]:
# Calculer MMD (Maximum Mean Discrepancy)
from evaluation_metrics.max_mean_discrepency import mmd_rbf
import numpy as np

mmd_scores = []
print("Calcul MMD par feature:\n")
print(f"{'Feature':<15} {'MMD':<10}")
print("-" * 25)

for feat_idx in range(c_real_data.shape[2]):
    real_feat = c_real_data[:, :, feat_idx].reshape(-1)
    gen_feat = c_gen_data[:, :, feat_idx].reshape(-1)
    
    mmd = mmd_rbf(real_feat, gen_feat)
    mmd_scores.append(mmd)
    print(f"{feature_names[feat_idx]:<15} {mmd:.6f}")

print("\n" + "="*25)
print(f"MMD moyen : {np.mean(mmd_scores):.6f}")
print("\n🎯 Cible : < 0.05 (excellent), < 0.10 (bon)")

In [ ]:
# Discriminative Score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Préparer données
X_real = c_real_data.reshape(len(c_real_data), -1)
X_gen = c_gen_data.reshape(len(c_gen_data), -1)

X = np.vstack([X_real, X_gen])
y = np.hstack([np.ones(len(X_real)), np.zeros(len(X_gen))])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Prédire
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nDiscriminative Score: {accuracy:.4f}")
print("🎯 Cible : ~0.50 (idéal = indistinguable)")

In [ ]:
# Créer rapport final
from datetime import datetime

report = f"""
{'='*60}
EHR-M-GAN TRAINING REPORT
{'='*60}

Date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Dataset: eICU-CRD Demo
GPU: {!nvidia-smi --query-gpu=name --format=csv,noheader | head -1}

{'='*60}
CONFIGURATION
{'='*60}
- Batch Size: {BATCH_SIZE}
- Pretraining Epochs: {NUM_PRE_EPOCHS}
- Training Epochs: {NUM_EPOCHS}
- Latent Dimension: {Z_DIM}
- Checkpoint Frequency: {CHECKPOINT_FREQ}

{'='*60}
DONNÉES
{'='*60}
- Patients traités: {len(c_real_data)}
- Fenêtre temporelle: 24 heures
- Features continues: {c_real_data.shape[2]} (vital signs)
- Features discrètes: {d_real_data.shape[2]} (interventions)

{'='*60}
MÉTRIQUES QUALITÉ
{'='*60}
- MMD moyen: {np.mean(mmd_scores):.6f} {'✅' if np.mean(mmd_scores) < 0.10 else '⚠️'}
- Discriminative Score: {accuracy:.4f} {'✅' if 0.45 <= accuracy <= 0.55 else '⚠️'}

MMD par feature:
"""

for fname, mmd in zip(feature_names, mmd_scores):
    report += f"  - {fname:<15}: {mmd:.6f}\n"

report += f"""
{'='*60}
FICHIERS GÉNÉRÉS
{'='*60}
- Checkpoints: {final_export}/checkpoints/
- Synthetic Data: {final_export}/synthetic_data/
- Logs: {final_export}/logs/
- Visualizations: {DRIVE_CHECKPOINT}/comparison_trajectories.png

{'='*60}
STATUT
{'='*60}
✅ Training complété avec succès
✅ Données synthétiques générées
✅ Sauvegarde Drive effectuée

{'='*60}
"""

print(report)

# Sauvegarder rapport
with open(f"{final_export}/TRAINING_REPORT.txt", "w") as f:
    f.write(report)

print(f"\n✅ Rapport sauvegardé : {final_export}/TRAINING_REPORT.txt")

## ✅ Checklist Finale

In [ ]:
# Vérifier que tout est sauvegardé
import os

checklist = [
    (f"{final_export}/checkpoints", "Checkpoints modèle"),
    (f"{final_export}/synthetic_data", "Données synthétiques"),
    (f"{final_export}/logs", "Logs training"),
    (f"{final_export}/TRAINING_REPORT.txt", "Rapport final"),
    (f"{DRIVE_CHECKPOINT}/comparison_trajectories.png", "Visualisations")
]

print("Vérification des fichiers:\n")
all_ok = True
for path, desc in checklist:
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"{status} {desc}: {path}")
    if not exists:
        all_ok = False

print("\n" + "="*60)
if all_ok:
    print("✅ TOUT EST SAUVEGARDÉ - Vous pouvez fermer le notebook")
else:
    print("⚠️  ATTENTION - Fichiers manquants, vérifiez les erreurs ci-dessus")

---

## 🎉 FIN DU TRAINING

### Prochaines étapes:

1. **Télécharger les résultats** depuis Drive
2. **Analyser les métriques** (MMD, Discriminative Score)
3. **Valider downstream** (modèles prédictifs)
4. **Publier résultats** (article, GitHub)

### Support:

- **Documentation**: [COLAB_SETUP_GUIDE.md](COLAB_SETUP_GUIDE.md)
- **GitHub**: https://github.com/jli0117/ehrMGAN
- **Article**: https://arxiv.org/abs/2112.12047

---

*Notebook créé par [Votre Nom] - Version 2.0 (Février 2026)*